# Search Engine

This lab is about starting from scratch, and hence having to structure the code yourself. A specification for the program you are to implement is given below - do pay attention as misunderstandings may cost you marks! Being precise is a necessary skill for a programmer. To give an executive summary, you are to code a search engine for recipes. A data set has been provided. The search engine is to be pretty basic, returning all recipes that contain all of the provided keywords. However, the user can choose from a number of orderings depending on their food preferences, which you need to support.

## Marking and submission

These lab exercises are marked, and contribute to your final grade. This lab exercise has 20 marks to earn, equivalent to 10% of your final grade.

Please submit your completed workbook to the auto marker before 8th November, 20:00 GMT. The workbook you submit must be an .ipynb file, which is saved into the directory you're running Jupyter; alternatively you can download it from the menu above using `File -> Download As -> Notebook (.ipynb)`. Remember to save your work regularly (`Save and checkpoint` in the `File` menu, the icon of a floppy disk, or `Ctrl-S`). It is wise to verify it runs to completion with _Restart & Run All_ before submission.

You must comply with the universities plagiarism guidelines: http://www.bath.ac.uk/library/help/infoguides/plagiarism.html

## Specification

The system must provide a function ``search``, with the following specification:
```
def search(query, ordering = 'normal', count = 10):
  ...
```

It `print`s out the results of the search, subject to the following rules:
1. It selects from the set of all recipes that contain __all__ of the words in the query (the positions/order of the words in the recipe are to be ignored).
2. It orders them based on the provided ordering (a string, meaning defined below).
3. It `print`s the top `count` matches only, preserving the order from best to worst. Must `print` just their title, one per line. Must be less than `count` if the search is specific enough that less than `count` recipes match.

As an aside, do not worry about memory usage. If duplicating some data can make your code faster/neater then feel free.



### Data set

A file, `recipes.json` is provided, containing 17K recipes. It can be parsed into a Python data structure using the [`json`](https://docs.python.org/3/library/json.html) module. It is a list, where each recipe is a dictionary containing various keys:
* `title` : Name of recipe; you can assume these are unique
* `categories` : A list of tags assigned to the recipe
* `ingredients` : What is in it, as a list
* `directions` : List of steps to make the recipe
* `rating` : A rating, out of 5, of how good it is
* `calories` : How many calories it has
* `protein` : How much protein is in it
* `fat` : How much fat is in it

Note that the data set was obtained via web scrapping and hence is noisy - every key except for `title` is missing from at least one recipe. Your code will need to cope with this.

You will probably want to explore the data before starting, so you have an idea of what your code has to deal with.

Data set came from https://www.kaggle.com/hugodarwood/epirecipes/version/2 though note it has been cleaned it up, by deleting duplicates and removing the really dodgy entries.



### Search

The search should check the following parts of the recipe (see data set description below):
* `title`
* `categories`
* `ingredients`
* `directions`

For instance, given the query "banana cheese" you would expect "Banana Layer Cake with Cream Cheese Frosting" in the results. Note that case is to be ignored ("banana" matches "Banana") and the words __do not__ have to be next to one another, in the same order as the search query or even in the same part of the recipe ("cheese" could appear in the title and "banana" in the ingredients). However, all words in the search query __must__ appear somewhere.



### Tokenisation

This is the term for breaking a sentence into each individual word (token). Traditionally done using regular expressions, and Python does have the `re` module, but there is no need to do that here (`re` can be quite fiddly). For matching words your tokenisation must follow the following steps:
1. Convert all punctuation and digits into spaces. For punctuation use the set in [`string.punctuation`](https://docs.python.org/3/library/string.html#string.punctuation), for digits [`string.digits`](https://docs.python.org/3/library/string.html#string.digits). You may find [`translate()`](https://docs.python.org/3/library/stdtypes.html#str.translate) interesting!
2. [`split()`](https://docs.python.org/3/library/stdtypes.html#str.split) to extract individual tokens.
3. Ignore any token that is less than $3$ characters long.
4. Make tokens lowercase.

When matching words for search (above) or ordering (below) it's only a match if you match an entire token. There are many scenarios where this simple approach will fail, but it's good enough for this exercise. The auto marker will be checking the above is followed! When doing a search the code should ignore terms in the search string that fail the above requirements.



### Ordering

There are three ordering modes to select from, each indicated by passing a string to the `search` function:
* `normal` - Based simply on the number of times the search terms appear in the recipe. A score is calculated and the order is highest to lowest. The score sums the following terms (repeated words are counted multiple times, i.e. "cheese cheese cheese" is $3$ matches to "cheese"):
    * $8 \times$ Number of times a query word appears in the title
    * $4 \times$ Number of times a query word appears in the categories
    * $2 \times$ Number of times a query word appears in the ingredients
    * $1 \times$ Number of times a query word appears in the directions
    * The `rating` of the recipe (if not available assume $0$)

* `simple` - Tries to minimise the complexity of the recipe, for someone who is in a rush. Orders to minimise the number of ingredients multiplied by the numbers of steps in the directions.

* `healthy` - Order from lowest to highest by this cost function:
$$\frac{|\texttt{calories} - 510n|}{510} + 2\frac{|\texttt{protein} - 18n|}{18} + 4\frac{|\texttt{fat} - 150n|}{150}$$
Where $n \in \mathbb{N}^+$ is selected to minimise the cost ($n$ is a positive integer and $n=0$ is not allowed). This can be understood in terms of the numbers $510$, $18$ and $150$ being a third of the recommended daily intake (three meals per day) for an average person, and $n$ being the number of whole meals the person gets out of cooking/making the recipe. So this tries to select recipes that neatly divide into a set of meals that are the right amount to consume for a healthy, balanced diet. Try not to overthink the optimisation of $n$, as it's really quite simple to do!

To clarify the use of the ordering string, to get something healthy that contains cheese you might call `search('cheese', 'healthy')`. In the case of a recipe that is missing a key in its dictionary the rules are different for each search mode:
* `normal` - Consider a missing entry in the recipe (e.g. no `ingredients` are provided) to simply mean that entry can't match any search words (because it has none!), but the item is still eligible for inclusion in the results, assuming it can match the search with a different entry.
* `simple` - If a recipe is missing either `ingredients` or `directions` it is dropped from such a search result. Because the data is messy if either of these lists is of length $1$ it should be assumed that the list extraction has failed and the recipe is to also be dropped from the search results.
* `healthy` - If any of `calories`, `protein` or `fat` is missing the recipe should be dropped from the result.



### Extra

You may find the [_inverted index_](https://en.wikipedia.org/wiki/Inverted_index) interesting. It's a data structure used by search engines. For each word a user may search for this contains a list of all documents (recipes) that contain the word. This may take a little effort to understand, but the resulting code will be both faster and neater.

## Advice

* Don't just start coding: Make a plan and work out what you intend to do.
* Think about structure, as messy code leads to mistakes.
* Plan your data structures. Don't be afraid to use sheets of paper if that works for you!
* Don't duplicate code, put it in a function/method instead.

* Divide and conquer. Break the system into parts that can implemented with minimal dependency on the rest. Functions or OOP are both suitable for doing this.
* Test early. Verify the individual parts work before trying to combine them. Factor this into the order you implement the parts of the system - don't implement something you are going to struggle to test before implementing, and verifying, other parts.
* Do not try and do a 'big bang', where you get it all working at once. Instead, get it working with features missing, then add those features in, one at a time.

* Keep things as simple as possible. Avoid long functions/methods.
* Include comments, as a form of planning and for your own sanity!
* Regularly reset the kernel and rerun the entire workbook. It is very easy to break something but not notice, because the correct version remains in memory.

## Marks
* __20 marks__: For many different _unit tests_ that will be run on `search`. They will cover all of the details in the above specification!
    * One test checks that it's faster than $0.1$ seconds on average (on the computer which runs the auto marker, which is quite fast) to do a search, so try not to be too inefficient (it ignores any time your notebook spends building data structures to be used by `search`). Note that the validation implementation comes in at $0.001$ seconds per search (after $5.5$ seconds of preparation), so this is very achievable!
    * You may want to look into Python's `set()` object, as it is useful for _one_ of the possible ways to implement this.
    * There will be sorting. The [Sorting how to](https://docs.python.org/3/howto/sorting.html) may help.
    * The auto marker does give some feedback, and you can run it as many times as you want. Don't be afraid to test an incomplete or semi-broken version of your code if stuck or unsure. It may help!
    * The validation implementation is 104 lines of code split over 5 cells (including white space for clarity and comments). Coded by someone who probably has much more experience than you, so you shouldn't aim to match this, but it's a good clue: If you find yourself at 500 lines you may want to stop and think some more! (line count does not include testing code, which is about the same amount again)

# Task 1 - Load the data

Work out a way to load the json file into a python data-structure

In [1]:
import json
import numpy as np
import string
import time
import unittest

In [2]:
with open('recipes.json') as json_file:
    data = json.load(json_file)
    


In [3]:

# '''Its important to note all of the info that is now in our dictionary! 
# For each entry we have dictionaries that correspond to
# - title[1]
# - categories[2]
# - ingredients[3]
# - directions[4]
# - ratings[5]
# - calories[6]
# - protein[7]
# - fat '''[8]




In [4]:
#The format right now is strange its a list of dictionaries, for ease i want to convert
#this into a dictionary of dictionaries

default = dict(title = [], categories = [], ingredients = [], directions = [], rating = 0, fat = 0, protein = 0, calories = 0)
def make_dict(data) :
    for key, value in default.items() :
        if key not in data :
            if key == 'categories' :
                data[key] = []
            elif key == 'ingredients' :
                data[key] = []
            elif key == 'directions' :
                data[key] = []
            elif key == 'rating' :
                data[key] = 0
            elif key == 'fat' :
                data[key] = 0
            elif key == 'protein' :
                data[key] = 0
            elif key == 'calories' :
                data[key] = 0
            else :
                data[key] = 0
    return data
    

# Tokenise the Query



Tokenisation


This is the term for breaking a sentence into each individual word (token). Traditionally done using regular expressions, and Python does have the re module, but there is no need to do that here (re can be quite fiddly). 

For matching words your tokenisation must follow the following steps:

Convert all punctuation and digits into spaces. For punctuation use the set in string.punctuation, for digits string.digits. You may find translate() interesting!

split() to extract individual tokens.

Ignore any token that is less than  3

Make tokens lowercase.



When matching words for search (above) or ordering (below) it's only a match if you match an entire token. 

There are many scenarios where this simple approach will fail, but it's good enough for this exercise. The 

auto marker will be checking the above is followed! When doing a search the code should ignore terms in the 
search string that fail the above requirements.


In [5]:
def tokenize(query):
# '''aim of this function is to tokenise the query we are going to use to search our data with
# input is our query, and an output is the tokenised version of this query'''
# This only however works for string inputs, ignoring lists and other data structures!
    a = string.punctuation
    b = string.digits
    punc_dict = {}
    for punc in a :
        punc_dict[punc] = ' '
    
    for digit in b :
        punc_dict[digit] = ' '
    
    # Convert unwanted string punctuation into blank spaces
    query = query.translate(query.maketrans(punc_dict))
    
    # convert unwanted numbers into blank spaces
    query = query.translate(query.maketrans(punc_dict))
    
    # Split on the spaces and make the characters all lower case
    tokens = query.lower().split(' ')
    
    
    #Only prints out words with length more than or equal to 3
    new_tokens = []
    for word in tokens :
        if len(word) >= 3 :
            new_tokens.append(word)
    return new_tokens

In [6]:
def tokenize_list(list) :
    #my tokenise function only works on single strings sentences but i want it to act on
    #lists within a dictionaries/lists
        tokens = []
        for word in list :
            tokens += tokenize(word)
            
        return tokens
    


In [7]:
# I wnat to tokenize the whole gaff
#We want to go throuhg the whole database
#turning that database into our new dictinoary form
#for each of those forms tokenize it! We dont want to however tokenize the 
#dictionaries for rating, calories, protein, fat.... numerical quantities


# TICK - converted whole data set into dictionary form
# TICK - tockenised the whole gafff!!!!!!!!
def organise(data) :
    m = len(data)
    recipes = []
    for i in range(m) :
        data[i] = make_dict(data[i])

        data[i]['categories'] = tokenize_list(data[i]['categories'])
        data[i]['length_ing'] = len(data[i]['ingredients'])
        data[i]['length_dir'] = len(data[i]['directions'])
        data[i]['ingredients'] = tokenize_list(data[i]['ingredients'])
        data[i]['directions'] = tokenize_list(data[i]['directions']) 
        data[i]['index'] = i
        
    return data

# Build Inverted Index

i.e this is preprocessing our data. Want to collate the words that appear multiple times by a score by alterring a dictionary.

I want something like this -
{Title : {word1 : #, word2 : #.....}, {categories : {word1 : #........}
we leave alone all quanitative values
we have already recorded the complexity of recipes by computing the length of ingredients and directions

In [8]:
def inverted_index(data) :

    n = len(data)

    new_data = []
    for i in range(n) :
        b = data[i]
        inverse_dict = {}
        for key in b :
            inverse_dict['new_title'] = b['title']
            if key == 'title' :
                for word in tokenize(b[key]) :

                    if word not in inverse_dict :
                        inverse_dict[word] = 8
                    else :
                        inverse_dict[word] += 8
            elif key == 'categories' :
                for word in b[key] :
                    if word not in inverse_dict :
                        inverse_dict[word] = 4
                    else :
                        inverse_dict[word] += 4
            elif key == 'ingredients' :
                for word in b[key] :
                    if word not in inverse_dict :
                        inverse_dict[word] = 2
                    else :
                        inverse_dict[word] += 2
            elif key == 'directions' :
                for word in b[key] :
                    if word not in inverse_dict :
                        inverse_dict[word] = 1
                    else :
                        inverse_dict[word] += 1
            else :
                inverse_dict[key] = b[key]
        new_data.append(inverse_dict)
    return new_data
            
                

# Ordering

## Normal Search

Orders depending on where the feature is found whether its title ingredietns.... and weights it accordingly!


In [9]:
def normal(query, lst) :
# '''takes in our query already tokenised and takes in a list of indices of our recipes
# our goal is to take this and to sort them according to their weights of which I included before'''
    before_sort_dict = {}
    #sorted dict is going to be a dict with this idndexing {'index' : 'score'}
    for i in lst :
        a = 0
        a += new_data[i]['rating'] 
        for word in query :
            a += new_data[i][word]
        before_sort_dict[i] = a
    sorted_dict = {k: v for k, v in sorted(before_sort_dict.items(), key=lambda item: item[1], reverse=True)}         
    return sorted_dict

## Simple Search

Aimed at people who in a rush and want to minimise directions and number of ingredients


If a recipe is missing either ingredients or directions it is dropped from such a search result. Because the data is messy if either of these lists is of length  1 it should be assumed that the list extraction has failed and the recipe is to also be dropped from the search results.

In [10]:
def simple(lst) :
# Again takes a list of matches nd aims to sort them depending on this simple weighting system!
    before_sort_dict = {}
    
    for i in lst :
        if new_data[i]['length_ing'] == 1 :
            continue
        elif new_data[i]['length_ing'] == 0 :
            continue
        elif new_data[i]['length_dir'] == 1 :
            continue
        elif new_data[i]['length_dir'] == 0 :
            continue
        else :
            score = new_data[i]['length_dir'] * new_data[i]['length_ing']
            before_sort_dict[i] = score
    sorted_dict = {k: v for k, v in sorted(before_sort_dict.items(), key=lambda item: item[1])}         
    return sorted_dict

            



## Healthy Search

In [11]:
def healthy(lst) :
    
    before_sort_dict = {}
    
    for i in lst :
        if new_data[i]['calories'] == 0 :
            continue
        elif new_data[i]['protein'] == 0 :
            continue
        elif new_data[i]['fat'] == 0 :
            continue
        else :
  
            cal = new_data[i]['calories']
            pro = new_data[i]['protein']
            fat = new_data[i]['fat']
            a = float(1/510)
            b = float(2/18)
            c = float(4/150)
            score = (a*np.absolute(cal - 510)) + (b * np.absolute(pro - 18)) + (c * np.absolute(fat - 150))
            before_sort_dict[i] = score


    sorted_dict = {k: v for k, v in sorted(before_sort_dict.items(), key=lambda item: item[1])} 
    return sorted_dict
        

In [12]:
def print_recipes(sorted_dict, count) :
    final_list = []
    for key in sorted_dict :
        final_list.append(new_data[key]['new_title'])
        
    print(*final_list[:count], sep='\n')

# Lets Search

I want to just create a basic search function that works through our recipe data structure and creates a list of all the recipes of interest
from there I will apply the sorting methods to catch the ones of interest

There are three ordering modes to select from, each indicated by passing a string to the search function:

In [13]:
data = organise(data)
new_data = inverted_index(data)

def search(query, ordering = 'normal', count = 10) :
    query = tokenize(query) 
    recipe_list = []
    recipe_index = []
    for i in range(len(new_data)) :
        if all(v in new_data[i] for v in query) :
            recipe_list.append(new_data[i]['new_title'])
            recipe_index.append(new_data[i]['index'])
            
            
    if ordering == 'normal' :
        a = normal(query, recipe_index)
#         print(*recipe_list, sep = '\n')
        print_recipes(a, count)
        
    elif ordering == 'simple' :
        a = simple(recipe_index)
        print_recipes(a,count)
        

    elif ordering == 'healthy' :
        a = healthy(recipe_index)
        print_recipes(a,count)      

In [14]:
search('pancakes', 'healthy', 5)

Scallion Potato Pancake and Smoked-Salmon Sandwiches 
Buttermilk Cornmeal Pancakes 
Shrimp and Green Onion Pancakes 
Fresh Corn Pancakes 
Orange Chocolate Chip Pancakes 


In [15]:
search('peanut butter', 'normal', 7)

Chocolate Cake with Milk Chocolate-Peanut Butter Frosting and Peanut Butter Brittle 
Peanut Butter Tart with Caramel-Peanut Glaze 
Peanut Butter Cake with Chocolate-Peanut Butter Icing 
Peanut Butter Banana Cream Pie 
Banana Cupcakes with Peanut Butter Frosting 
Chocolate-Peanut Butter Cake with Cream Cheese and Butterfinger Frosting 
Peanut Butter Cheesecake with Peanut Brittle 


In [16]:
search('lemon honey cake', 'normal', 4)

Lemon-Honey Almond Cake 
Chocolate, Orange and Honey Cake 
Pear Cake with Lemon-Honey Cream Cheese Frosting 
Lemon Buttermilk Cake 


In [17]:
search('mousse', 'simple', 1)

Simplest Chocolate Honey Mousse 


In [18]:
search('baked potato', 'simple', 3)

Tiny Baked Potatoes with Chervil Cream 
Baked Whole Fish with Potatoes and Lemon 
Twice-Baked Potatoes with Basil and Sour Cream 


In [19]:
start= time.time()
search('pie', 'healthy', 20)

end = time.time()
print(end - start)

Corn and Bacon Pie 
Spinach, Red Pepper, and Feta Quiche 
Zucchini, Sun-Dried Tomato, and Mozarella Tart 
Flaky Pie Crust Dough 
Chocolate Mousse Pie 
Bacon and Green Chili Quiche 
Yellow Squash and Mozzarella Quiche with Fresh Thyme 
Scallop and Shrimp Fritters with Chipotle Mayonnaise 
Butter Pie Crust 
Challah, Sausage, and Dried Cherry Stuffing 
Frozen Peanut Butter Pie with Candied Bacon 
Butter Pie Crust Dough 
Warm Goat Cheese Salad with Pears and Walnuts 
Kale, Butternut Squash, and Pancetta Pie 
Asparagus Tart 
Flaky Pie Crust 
Caramelized Garlic, Spinach, and Cheddar Tart 
Pear and Almond Tart 
Bacon and Potato Pie 
Sweet Cheese Pie 
0.041416168212890625


In [20]:
search('blue cheese salad', 'healthy', 10)

Iceberg with Tomatoes, Blue Cheese, and Bacon 
Bacon and Blue Cheese Salad with Caesar Dressing 
Blue Cheese Caesar Salad 
Salad Bar Cobb 
Roast Beef Salad with Cabbage and Horseradish 
Irish Pub Salad 
Curly Endive Salad with Cranberry-Bacon Vinaigrette and Blue Cheese Pastries 
Heirloom Tomato Salad with Blue Cheese 
Vegetable and Wild Rice Salad 
Beefsteak Tomato, Butterhead Lettuce, and Bacon with Blue Cheese Dressing 


In [21]:
search('tempura fish', 'healthy', 10)

In [22]:
search('leek blue cheese cream cheese', 'healthy', 3)

Spinach and Leek Gratin with Roquefort Crumb Topping 
Cream of Cauliflower and Stilton Soup 
Veal with Leek and Roquefort Sauce 


In [23]:
search('curry, vegetable', 'healthy', 3)

Red Curry Peanut Sauce 
Shrimp Curry with Yu Choy and Kabocha Squash 
Shrimp Curry 


In [26]:
search('sausages, flour')

Seafood and Turkey-Sausage Gumbo 
Steak House Creamed Spinach 
Calzones with Cheese, Sausage and Roasted Red Pepper 
Chicken with Parsley and Thyme Stuffing and Roasted Potatoes 
Lentil Ravioli with Sausage and Yellow Tomato Sauce 
Double-Crusted Turkey Sausage Pizza 
Golden Crisp Daikon Cake with Spicy Herb Soy Sauce 
Spicy Gumbo-Laya 
Chorizo, Poblano and Yam Fajitas with Lime-Marinated Red Onions 
Mixed-Greens and Sausage Soup with Cornmeal Dumplings 
